# Получение рекомендаций в Spotify с помощью GigaChat

Этот ноутбук показывает, как можно получать рекомендаций в Spotify c помощью GigaChat.
Получить Client Id и Client Secret в Spotify можно [тут](https://developer.spotify.com/dashboard)

## Установка зависимостей

Для работы примера установите зависимости:

In [ ]:
!pip install spotipy langchain-gigachat python-dotenv -q

## Инициализация GigaChat

Для работы с моделями GigaChat, нужно передать ключ авторизации GigaChat API.

Ключ можно указать в переменной среды `GIGACHAT_CREDENTIALS`, заданной в файле `.env` или созданной с помощью команды:

```sh
export GIGACHAT_CREDENTIALS=ключ_авториазации
```

При инициализации проверяется наличие переменной среды `GIGACHAT_CREDENTIALS` с заданным ключом авторизации GigaChat API.
Если переменная отсутствует, вы сможете указать ключ в поле **Введите ключ авторизации GigaChat API**.

О способах авторизации и поддерживаемых переменных среды — в [README библиотеки gigachat](https://github.com/ai-forever/gigachat).

In [ ]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(
    model="GigaChat-Pro",
    verify_ssl_certs=False
)

## Авторизация в Spotify

Создайте переменные `SP_ID` и `SP_SECRET` со своими учетными данными, чтобы авторизоваться в Spotify:

In [6]:
import json
import sys
import spotipy

from langchain.agents import AgentExecutor, ZeroShotAgent
from langchain.tools import Tool
from spotipy.oauth2 import SpotifyClientCredentials

os.environ["SP_ID"] = "<Spotify Client ID>"
os.environ["SP_SECRET"] = "<Spotify Client Secret>"

spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id=os.getenv("SP_ID"), client_secret=os.getenv("SP_SECRET")
    )
)

In [9]:
def create_playlist(data):
    """Функция получения рекомендаций"""
    data = json.loads(data)
    kwargs = {}
    if "energy" in data:
        kwargs["target_energy"] = data["energy"] / 100
    if "happiness" in data:
        kwargs["target_valence"] = data["happiness"] / 100
    if "dance" in data:
        kwargs["target_danceability"] = data["dance"] / 100
    if "genres" in data:
        kwargs["seed_genres"] = data["genres"][:5]
    else:
        kwargs["seed_genres"] = ["pop", "hip-hop", "rock", "r-n-b"]

    results = spotify.recommendations(limit=10, **kwargs)
    tracks = []
    for track in results["tracks"]:
        name = f"{track['artists'][0]['name']} — {track['name']}"
        os.write(1, f"{name}\n".encode())
        tracks.append(name)

    return "\n".join(tracks)


tools = [
    Tool.from_function(
        func=create_playlist,
        return_direct=True,
        name="CREATE_PLAYLIST",
        description="""Описание: создает плейлисты \n Параметры:
        {{"happiness": число которое обозначает настроение запроса где 0 - грусть, печаль, агрессивность и 100 - счастье, веселье
         "energy": число которое обозначает энергичность запроса где 0 — спокойствие и 100 — энергичность не обязательный параметр,
         "dance": число которое обозначает танцевальность где 0 - спокойствие и 100 — танцевальная музыка не обязательный параметр,
         "genres": список от 1 до 5 элементов выбранных из этого списка ["acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-dubstep", "power-pop", "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "rainy-day", "reggae", "reggaeton", "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo", "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish", "study", "summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", "work-out", "world-music"]}}''
         """,  # noqa: E501,
    ),
]

agent = ZeroShotAgent.from_llm_and_tools(llm=llm, tools=tools)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

stdout = sys.stdout

sys.stdout = stdout

print(
    agent_executor.run(
        "Создай мне грустный плейлист, когда хочется сбежать от всех проблем"
    )
)



> Entering new AgentExecutor chain...
-----
Thought: Создать плейлист с настроением грусть и жанрами, которые подходят для такого настроения
Action: CREATE_PLAYLIST
Action Input: {"happiness": 0, "genres": ["sad", "ballads", "love-songs"]}

Observation: John Murphy — In the House - In a Heartbeat
Lana Del Rey — Carmen
Céline Dion — My Heart Will Go On (Love Theme from "Titanic")
Lifehouse — Everything
Touché Amoré — Condolences
Of Mice & Men — My Understandings
Coldplay — Warning Sign
Sigur Rós — Untitled #1 (Vaka)
Troye Sivan — Happy Little Pill
B.o.B — Both of Us (feat. Taylor Swift)


> Finished chain.
John Murphy — In the House - In a Heartbeat
Lana Del Rey — Carmen
Céline Dion — My Heart Will Go On (Love Theme from "Titanic")
Lifehouse — Everything
Touché Amoré — Condolences
Of Mice & Men — My Understandings
Coldplay — Warning Sign
Sigur Rós — Untitled #1 (Vaka)
Troye Sivan — Happy Little Pill
B.o.B — Both of Us (feat. Taylor Swift)
